# Itinerary Basics

Basic report on stage itinerary - stage name, distance etc.

In [1]:
import notebookimport

In [2]:
import os
import sqlite3
import pandas as pd

In [3]:
#!pip3 install tracery
#!pip3 install inflect

In [8]:
if __name__=='__main__':
    #dbname='wrc18.db'
    YEAR=2019
    dbname='sweden19.db'
    conn = sqlite3.connect(dbname)
    rally='Sweden'
    rebase = 'LOE'
    rebase = ''

In [9]:
#!/Users/ajh59/anaconda3/bin/pip install tracery
import tracery
from tracery.modifiers import base_english
from inflectenglish import inflect_english
from pytracery_logic import pytracery_logic

def pandas_row_mapper(row, rules, root,  modifiers=base_english):
    ''' Generate text from a single row of dataframe using a tracery grammar. '''
    row=row.to_dict()
    rules=rules.copy()

    for k in row:
        rules[k] = str(row[k])
        grammar = tracery.Grammar(rules)
        if modifiers is not None:
            if isinstance(modifiers,list):
                for modifier in modifiers:
                    grammar.add_modifiers(modifier)
            else:
                grammar.add_modifiers(modifiers)

    return grammar.flatten(root)

def traceryparse(rules, root, modifiers=base_english):
    ''' Create a tracery grammar from a rules set and set of modifiers and apply it to tracery template string. '''
    grammar = tracery.Grammar(rules)
    if modifiers is not None:
        if isinstance(modifiers,list):
            for modifier in modifiers:
                grammar.add_modifiers(modifier)
        else:
            grammar.add_modifiers(modifiers)
    
    return grammar.flatten(root)
    

## Rally Summary

In [10]:
def dbGetRallySummary(rally, year=YEAR):
    ''' Create a dataframe containing summary details of a specified rally. '''
    q='''
    SELECT o.*, t.totalDistance FROM 
        (SELECT ce.`country.name`, ce.startDate, ce.finishDate, ce.name, ce.`country.iso3`, ce.surfaces,
            COUNT(*) numOfStages, SUM(itc.distance) AS compDistanceKm
            FROM itinerary_controls itc
            JOIN championship_events ce ON itc.eventId=ce.eventId
            WHERE ce.`country.name`="{rally}" AND type='StageStart' AND strftime('%Y', startDate)='{year}' ) AS o
        JOIN (SELECT ce.`country.name`, SUM(itc.distance) totalDistance FROM itinerary_controls itc
                JOIN championship_events ce ON itc.eventId=ce.eventId
                WHERE ce.`country.name`="{rally}") AS t ON o.`country.name` = t.`country.name`
    '''.format(rally=rally, year=year)
    
    rallydetails = pd.read_sql(q,conn)
    
    return rallydetails


In [11]:
if __name__=='__main__':
    rs = dbGetRallySummary(rally)
    display(rs)
    display(rs.dtypes)

,country.name,startDate,finishDate,name,country.iso3,surfaces,numOfStages,compDistanceKm,totalDistance
0,Sweden,2019-02-14,2019-02-17,Rally Sweden,SWE,Snow,19,316.8,1460.59


country.name       object
startDate          object
finishDate         object
name               object
country.iso3       object
surfaces           object
numOfStages         int64
compDistanceKm    float64
totalDistance     float64
dtype: object

Tracery rules specify an emergent story template.

In [12]:
rules = {'origin': "#name# (#startDate# to #finishDate#) #stages#. #dist#. #surface#.",
         'stages': "runs over #numOfStages# competitive special stages",
         'dist': "The distance covered on the special stages is #compDistanceKm.round#km, with an overall rally distance of #totalDistance.round#km",
         'surface':"The special stage surface type is predominantly #surfaces#",
        }

In [13]:
if __name__=='__main__':
    rs['report'] = rs.apply(lambda row: pandas_row_mapper(row, rules, "#origin#",[base_english,inflect_english, pytracery_logic]), axis=1)

In [14]:
if __name__=='__main__':
    rs[rs['country.name']==rally]['report'].iloc[0]

'Rally Sweden (2019-02-14 to 2019-02-17) runs over 19 competitive special stages. The distance covered on the special stages is 316.80km, with an overall rally distance of 1460.59km. The special stage surface type is predominantly Snow.'

In [15]:
basepath = 'report'
if not os.path.exists(basepath):
    os.makedirs(basepath)

In [16]:
README='''# Rally Report - {}

*This report is unofficial and is not associated in any way with the Fédération Internationale de l’Automobile (FIA) or WRC Promoter GmbH.*


{}
'''.format(rally, rs[rs['country.name']==rally]['report'].iloc[0])

with open('{}/README.md'.format(basepath), 'w') as out_file:
    out_file.write(README)

In [17]:
import inflect
p=inflect.engine()

In [18]:
rs['startDate']=pd.to_datetime(rs['startDate'])
rs['startDate'].loc[0].strftime("%A %d,  %B %Y")

'Thursday 14,  February 2019'

In [19]:
p.number_to_words(p.ordinal((int(rs['startDate'].loc[0].strftime("%d")))))

'fourteenth'

## Itinerary Items

In [20]:
q='''
    SELECT *
    FROM itinerary_controls itc
    JOIN championship_events ce ON itc.eventId=ce.eventId
    JOIN itinerary_sections isc ON itc.`itinerarySections.itinerarySectionId`=isc.itinerarySectionId
    JOIN itinerary_legs il ON isc.itineraryLegId=il.itineraryLegId
    WHERE ce.`country.name`="{rally}" AND strftime('%Y', startDate)='{year}' 
            AND firstCarDueDateTimeLocal NOT NULL ORDER BY firstCarDueDateTimeLocal 
    '''.format(rally=rally, year=YEAR)
xx =pd.read_sql(q,conn)
xx.columns

Index(['code', 'controlId', 'controlPenalties', 'distance', 'eventId',
       'firstCarDueDateTime', 'firstCarDueDateTimeLocal', 'location',
       'stageId', 'status', 'targetDuration', 'targetDurationMs',
       'timingPrecision', 'type', 'itineraryLegId',
       'itinerarySections.itinerarySectionId', 'roundingPolicy', 'categories',
       'clerkOfTheCourse', 'country.countryId', 'country.iso2', 'country.iso3',
       'country.name', 'countryId', 'eventId', 'finishDate', 'location',
       'mode', 'name', 'organiserUrl', 'slug', 'startDate', 'stewards',
       'surfaces', 'templateFilename', 'timeZoneId', 'timeZoneName',
       'timeZoneOffset', 'trackingEventId', 'itineraryLegId',
       'itinerarySectionId', 'name', 'order', 'itineraryId', 'itineraryLegId',
       'legDate', 'name', 'order', 'startListId', 'status'],
      dtype='object')

In [21]:
xx.head()

,code,controlId,controlPenalties,distance,eventId,firstCarDueDateTime,firstCarDueDateTimeLocal,location,stageId,status,...,itinerarySectionId,name,order,itineraryId,itineraryLegId,legDate,name,order,startListId,status
0,TC1,3818,All,NaN,79,2019-02-14T19:04:00,2019-02-14T20:04:00+01:00,Start Färjestad Trotting Course Karlstad,901,Completed,...,368,Section 1,1,148,160,2019-02-14,Thursday 14th February,1,267,Completed
1,SS1,3819,None,1.90,79,2019-02-14T19:08:00,2019-02-14T20:08:00+01:00,Super Special Stage Karlstad 1,901,Completed,...,368,Section 1,1,148,160,2019-02-14,Thursday 14th February,1,267,Completed
2,TC1A,3782,Late,102.33,79,2019-02-14T20:58:00,2019-02-14T21:58:00+01:00,Parc Fermé Torsby IN,-1,Completed,...,368,Section 1,1,148,160,2019-02-14,Thursday 14th February,1,267,Completed
3,TC1B,3781,All,NaN,79,2019-02-15T05:00:00,2019-02-15T06:00:00+01:00,Parc Fermé Torsby OUT/Service IN,-1,Completed,...,369,Section 2,2,148,159,2019-02-15,Friday 15th February,2,268,Completed
4,TC1C,3780,All,NaN,79,2019-02-15T05:15:00,2019-02-15T06:15:00+01:00,Service OUT,-1,Completed,...,369,Section 2,2,148,159,2019-02-15,Friday 15th February,2,268,Completed


In [22]:
import datetime
t=datetime.time(abs(int(360/60)), 360 % 60)
dt = datetime.datetime.combine(datetime.date.today(), t)
dt.isoformat()

'2019-02-17T06:00:00'

In [23]:
t = datetime.datetime.combine(datetime.date.today(),datetime.time(0))
(t  + datetime.timedelta( minutes=-359)).isoformat()

'2019-02-16T18:01:00'

In [24]:
xx[:2][['timeZoneId','timeZoneOffset']]

,timeZoneId,timeZoneOffset
0,W. Europe Standard Time,60
1,W. Europe Standard Time,60


In [25]:
def dbGetTimeControls(rally, year=YEAR):
    ''' Get dataframe containing time control details for a specified rally. '''
    q='''
    SELECT il.name AS date, itc.*, ce.timeZoneOffset,
         isc.itinerarySectionId, isc.name AS section, isc.`order`
    FROM itinerary_controls itc
    JOIN championship_events ce ON itc.eventId=ce.eventId
    JOIN itinerary_sections isc ON itc.`itinerarySections.itinerarySectionId`=isc.itinerarySectionId
    JOIN itinerary_legs il ON isc.itineraryLegId=il.itineraryLegId
    WHERE ce.`country.name`="{rally}" AND strftime('%Y', startDate)='{year}'
            AND firstCarDueDateTimeLocal NOT NULL ORDER BY firstCarDueDateTimeLocal 
    '''.format(rally=rally, year=year)
    time_controls = pd.read_sql(q,conn)
    time_controls['firstCarDueDateTimeLocal']=pd.to_datetime(time_controls['firstCarDueDateTimeLocal'])
    return time_controls

In [26]:
if __name__=='__main__':
    time_controls = dbGetTimeControls(rally)
    display(time_controls.head())

,date,code,controlId,controlPenalties,distance,eventId,firstCarDueDateTime,firstCarDueDateTimeLocal,location,stageId,...,targetDurationMs,timingPrecision,type,itineraryLegId,itinerarySections.itinerarySectionId,roundingPolicy,timeZoneOffset,itinerarySectionId,section,order
0,Thursday 14th February,TC1,3818,All,NaN,79,2019-02-14T19:04:00,2019-02-14 20:04:00+01:00,Start Färjestad Trotting Course Karlstad,901,...,NaN,Minute,TimeControl,160,368,NoRounding,60,368,Section 1,1
1,Thursday 14th February,SS1,3819,None,1.90,79,2019-02-14T19:08:00,2019-02-14 20:08:00+01:00,Super Special Stage Karlstad 1,901,...,240000.0,Minute,StageStart,160,368,RoundToClosestMinute,60,368,Section 1,1
2,Thursday 14th February,TC1A,3782,Late,102.33,79,2019-02-14T20:58:00,2019-02-14 21:58:00+01:00,Parc Fermé Torsby IN,-1,...,6600000.0,Minute,TimeControl,160,368,NoRounding,60,368,Section 1,1
3,Friday 15th February,TC1B,3781,All,NaN,79,2019-02-15T05:00:00,2019-02-15 06:00:00+01:00,Parc Fermé Torsby OUT/Service IN,-1,...,NaN,Minute,TimeControl,159,369,NoRounding,60,369,Section 2,2
4,Friday 15th February,TC1C,3780,All,NaN,79,2019-02-15T05:15:00,2019-02-15 06:15:00+01:00,Service OUT,-1,...,900000.0,Minute,TimeControl,159,369,NoRounding,60,369,Section 2,2


In [27]:
#Check datetime type
time_controls['firstCarDueDateTime'] = pd.to_datetime(time_controls['firstCarDueDateTime'])

In [28]:
import datetime

def newtime(row):
    t=datetime.timedelta( minutes=row['timeZoneOffset'])
    return row['firstCarDueDateTime']+ t

time_controls['mylocaltime'] = time_controls.apply(lambda row: newtime(row),axis=1)

In [29]:
rules = {'origin': "#mylocaltime.pdtime(%H:%M:%S)# #code# #location# #distance.isNotNull(post=km).brackets# \[#targetDuration#\]",
        }

In [30]:
dategroups = time_controls.groupby('date',sort=False)
for key in dategroups.groups.keys():
    print('---\n\n{}:\n'.format(key))
    grouped2=dategroups.get_group(key).groupby('section',sort=False)
    for key2 in grouped2.groups.keys():
        g2 = grouped2.get_group(key2)
        l=len(g2[g2['code'].str.startswith('SS')])
        print('{} - {} special {}\n'.format(key2,p.number_to_words(l),p.plural_noun('stage',l)))
        for r in grouped2.get_group(key2).apply(lambda row: pandas_row_mapper(row, rules, "#origin#",[base_english,inflect_english]), axis=1):
            print('\t{}'.format(r))
        print('\n')

---

Thursday 14th February:

Section 1 - one special stage

	20:04:00 TC1 Start Färjestad Trotting Course Karlstad  [None]
	20:08:00 SS1 Super Special Stage Karlstad 1 (1.9km) [00:04:00]
	21:58:00 TC1A Parc Fermé Torsby IN (102.33km) [01:50:00]


---

Friday 15th February:

Section 2 - three special stages

	06:00:00 TC1B Parc Fermé Torsby OUT/Service IN  [None]
	06:15:00 TC1C Service OUT  [00:15:00]
	07:52:00 TC2 Hof-Finnskog (92.23km) [01:37:00]
	07:55:00 SS2 Hof-Finnskog 1 (21.26km) [00:03:00]
	09:05:00 TC3 Svullrya (44.79km) [01:10:00]
	09:08:00 SS3 Svullrya 1 (24.88km) [00:03:00]
	09:56:00 TC4 Röjden (16.44km) [00:48:00]
	09:59:00 SS4 Röjden 1 (18.1km) [00:03:00]
	11:19:00 TC4A Regroup and Technical Zone IN (52.79km) [01:20:00]


Section 3 - four special stages

	11:34:00 TC4B Regroup OUT/Service IN  [00:15:00]
	12:14:00 TC4C Service OUT  [00:40:00]
	13:51:00 TC5 Hof-Finnskog (92.23km) [01:37:00]
	13:54:00 SS5 Hof-Finnskog 2 (21.26km) [00:03:00]
	15:14:00 TC6 Svullrya (44.79km) [

In [31]:
def initStageReports(rebase='overall'):
    ''' Generate the initial report for a particular section rebased to a particular position or driver.'''
    sectionREADME = '''### {section} Report
    '''

    SUMMARY ='\n'
    
    rules['stages'] = "#code# - #location# #distance.isNotNull(post=km).brackets#"

    dn = '' if not rebase or 'overall' in rebase else '_'+rebase
  
    sections = time_controls.groupby('section',sort=False)
    keyorder = [k for k in time_controls.sort_values('order')['section'].unique() if k in sections.groups.keys()]
    for key in keyorder:

        sectionfn = '{bp}/{key}_report{dn}.md'.format(bp=basepath,key=key, dn=dn)

        with open(sectionfn, 'w') as out_file:
            out_file.write('')

        sectionControls = sections.get_group(key)
        sstages = sectionControls[sectionControls['code'].str.startswith('SS')]
        l=len(sstages)
        
        if l:
            title = '# {}, {}\n\nThis section comprises {} special {}'.format(key,sectionControls['date'].iloc[0], p.number_to_words(l),p.plural_noun('stage',l))
            sstage=[]
            for r in sstages.apply(lambda row: pandas_row_mapper(row, rules, "#stages#",[base_english,inflect_english]), axis=1):
                sstage.append(r)
            sectionREADME = '{} ({})'.format(sectionREADME,', '.join(sstage))
        else: 
            title = '# {}, {}\n\nThere were no special stages in this section.'.format(key,sectionControls['date'].iloc[0])
        sectionREADME = '''{title}'''.format( title=title)

        sectionREADME = '''{s}\n\nThe full scheduled itinerary for the section was as follows:\n'''.format(s=sectionREADME,)

        controls = []
        for r in sectionControls.apply(lambda row: pandas_row_mapper(row, rules, "#origin#",[base_english,inflect_english]), axis=1):
            controls.append(r)
        sectionREADME = '{}\n\t- {}\n'.format(sectionREADME, '\n\t- '.join(controls))

        print(sectionREADME,'\n------\n')
        with open(sectionfn, 'a') as out_file:
            out_file.write(sectionREADME)

        #Add section
        SUMMARY = '{summary}\n* [{key}]({key}_report{dn}.md)\n'.format(summary=SUMMARY, key=key,dn=dn)

        sstageDict = sstages.set_index('code').to_dict(orient='index')
        #Add special stages
        for s in sstageDict: #[Section 2](Section 2_report.md)
            SUMMARY = '{summary}  - [{s} - {n}]({s}_report{dn}.md)\n'.format(summary=SUMMARY,
                                                                              s=s, 
                                                                              n=sstageDict[s]['location'],
                                                                              dn=dn)

    with open('{}/SUMMARY{}.md'.format(basepath,'_{}'.format(dn)), 'a') as out_file:
        out_file.write(SUMMARY)

initStageReports(rebase)
#initStageReports()

# Section 1, Thursday 14th February

This section comprises one special stage

The full scheduled itinerary for the section was as follows:

	- 20:04:00 TC1 Start Färjestad Trotting Course Karlstad  [None]
	- 20:08:00 SS1 Super Special Stage Karlstad 1 (1.9km) [00:04:00]
	- 21:58:00 TC1A Parc Fermé Torsby IN (102.33km) [01:50:00]
 
------

# Section 2, Friday 15th February

This section comprises three special stages

The full scheduled itinerary for the section was as follows:

	- 06:00:00 TC1B Parc Fermé Torsby OUT/Service IN  [None]
	- 06:15:00 TC1C Service OUT  [00:15:00]
	- 07:52:00 TC2 Hof-Finnskog (92.23km) [01:37:00]
	- 07:55:00 SS2 Hof-Finnskog 1 (21.26km) [00:03:00]
	- 09:05:00 TC3 Svullrya (44.79km) [01:10:00]
	- 09:08:00 SS3 Svullrya 1 (24.88km) [00:03:00]
	- 09:56:00 TC4 Röjden (16.44km) [00:48:00]
	- 09:59:00 SS4 Röjden 1 (18.1km) [00:03:00]
	- 11:19:00 TC4A Regroup and Technical Zone IN (52.79km) [01:20:00]
 
------

# Section 3, Friday 15th February

This section compri

In [109]:
sstages['date']

NameError: name 'sstages' is not defined

In [110]:
sstages.set_index('code').to_dict(orient='index')

NameError: name 'sstages' is not defined

In [41]:
print(SUMMARY)

NameError: name 'SUMMARY' is not defined

In [111]:
!head -n100 "report/Section 3_report.md"

# Section 3, Friday 25th January

This section comprises three special stages

The full scheduled itinerary for the section was as follows:

	- 12:27:00 TC5B Technical Zone and Regroup (OUT) - Service (IN) (0.1km) [00:10:00]
	- 13:00:00 TC5C Service (OUT) (1.3km) [00:33:00]
	- 14:20:00 TC6 VALDROME (69.74km) [01:20:00]
	- 14:23:00 SS6 VALDROME - SIGOTTIER 2 (20.04km) [00:03:00]
	- 15:23:00 TC7 ROUSSIEUX (33.16km) [01:00:00]
	- 15:26:00 SS7 ROUSSIEUX - LABOREL 2 (24.05km) [00:03:00]
	- 16:46:00 TC8 CURBANS (47.29km) [01:20:00]
	- 16:49:00 SS8 CURBANS - PIEGUT 2 (18.47km) [00:03:00]
	- 17:34:00 TC8A Technical Zone (IN)  (13.73km) [00:45:00]
	- 17:44:00 TC8B Technical Zone (OUT) - Flexi Service (IN) (0.1km) [00:10:00]
	- 18:32:00 TC8C Flexi Service (OUT) - Parc Fermé (IN)  (1.3km) [00:48:00]


In [ ]:
sectionControls['code']

In [ ]:
sectionControls#[sectionControls['code']]

In [ ]:
l